Este proyecto tiene como objetivo desarrollar un modelo de procesamiento
de lenguaje natural (NLP) capaz de generar resúmenes clínicos automáticos
a partir de un dataset de alrededor de 1700 conversaciones entre doctores y
sus pacientes, junto con los respectivos resúmenes y anotaciones.

    Los objetivos de esta entrega 3 son:

    1. Correciones de la entrega 2:

Por ejemplo, ¿por qué lematizais? ¿Habéis analizado qué pasa con los word embeddings lematizado vs no-lematizado? Para embeddings la recomendación es no lematizar y con tf-idf habría que analizarlo con la tarea que queráis resolver. Además, ¿qué son lo que vosotros denomiáis tokens? Porque de 4367 palabras únicas no sé cómo salen 173,867 tokens.

Por otro lado, el análisis de longitud está muy bien pero lo hacéis a nivel de palabra, no de token. De cara a siguientes entregas hacerlo también a nivel de token para ver si un BIOBert por ejemplo tiene contexto suficiente.



    2. Definición de la tarea:
Generación de resúmenes y clasificación del diagnóstico.

    3. Tareas Específicas de la entrega 3

Para ello, se deberán usar técnicas tanto de Shallow ML (o ML tradicional), como algunos de los modelos de CNNs o Redes Recurrentes que hemos visto en clase.

Comparar experimentos usando distintas métricas y optimizar los hiperparámetros.

Usar atención, combinar features (no creo que aplique a nuestro problema)

    Mínimos exigibles:
Dos técnicas de Shallow Learning utilizando técnicas de representación dispersa/sparse.

Dos técnicas de Deep Learning comparando diferentes tipos de embeddings y fine-tuneandolos dependiendo del caso. Ejemplos:

Word2Vec congelado vs Word2Vec fine-tuneado vs Word2Vec “from scratch”

Embedding fine-tuneado durante el entrenamiento vs Embedding inicializado

Comparar al menos dos formas de embeddings de cada tipo:

Tradicionales: e.g., Bag-of-Words, TF-IDF, etc.

Semánticos No-Contextuales: e.g., Glove, FastText, Word2Vec, etc.

Contextuales: e.g., ELMo, BERT, Modelos pre-entrenados de Hugging-Face, etc.

In [ ]:
# Instalación de librerías

%pip install -q spacy gensim transformers torch tensorflow tensorflow-hub seaborn matplotlib scikit-learn

In [5]:
# Descargar modelo de idioma de spaCy

%pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_md-0.5.4.tar.gz


  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_md-0.5.4.tar.gz (119.1 MB)
  Preparing metadata (setup.py) ... done
  Using cached spacy-3.7.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (27 kB)
  Using cached thinc-8.2.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (15 kB)
  Using cached typer-0.20.0-py3-none-any.whl.metadata (16 kB)
  Using cached langcodes-3.5.0-py3-none-any.whl.metadata (29 kB)
  Using cached numpy-2.3.5-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached language_data-1.3.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached blis-0.7.11-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.4 kB)
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached marisa_trie-1.3.1-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (10 kB)
Using cached 

In [8]:





# Imports
import re
import pickle
import unicodedata
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# NLP y embeddings
import spacy
from gensim.models import Word2Vec
import gensim.downloader as api
from transformers import AutoTokenizer, AutoModel

# Machine Learning
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Deep Learning
import torch
import tensorflow as tf
import tensorflow_hub as hub

/home/bnnyrabbit/Uni/4.o/NLP/.env_nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-18 20:03:31.589629: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/bnnyrabbit/Uni/4.o/NLP/.env_nlp/lib/python3.12/site-packages/tensorflow_hub/__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [9]:
df = pd.read_csv("dataset/MTS-Dialog-TrainingSet.csv")

In [10]:
def normalize_text(s, lowercase=True):
    if pd.isna(s):
        return ""
    # Normalizar unicode
    s = unicodedata.normalize("NFKC", str(s))
    # Marcadores de quién habla
    s = re.sub(r'\bDoctor[:\-]\s*', ' <DOC> ', s, flags=re.I)
    s = re.sub(r'\bPatient[:\-]\s*', ' <PAT> ', s, flags=re.I)
    # Espacios
    s = re.sub(r'\s+', ' ', s).strip()
    # Lowercase opcional
    if lowercase:
        s = s.lower()
    return s

# Versión para ELMo (lowercase)
df['dialog_clean'] = df['dialogue'].apply(lambda x: normalize_text(x, lowercase=True))

# Versión para BIO/ClinicalBERT (manteniendo mayúsculas)
df['dialog_clean_clinicBERT'] = df['dialogue'].apply(lambda x: normalize_text(x, lowercase=False))

# Los resúmenes
df['section_text_clean'] = df['section_text'].apply(lambda x: normalize_text(x, lowercase=True))

In [11]:
# Embeddings (sin lemmatization)

w2v = api.load("word2vec-google-news-300")          # Word2Vec




[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [13]:
glove = api.load("glove-wiki-gigaword-300")        # GloVe


[==================================================] 100.0% 376.1/376.1MB downloaded


In [14]:
ft = api.load("fasttext-wiki-news-subwords-300")   # FastText

[==================================================] 100.0% 958.5/958.4MB downloaded


In [15]:
# Obtener vocabulario del dataset
all_tokens = []
for text in df['dialog_clean']:
    if pd.notna(text):
        all_tokens.extend(text.split())

vocab = set(all_tokens)
vocab_freq = Counter(all_tokens)

print(f"\nVocabulario total: {len(vocab):,} palabras únicas")
print(f"Total de tokens: {len(all_tokens):,}")

# Análisis para cada modelo
models = {
    'Word2Vec (Google News)': w2v,
    'GloVe (Wiki Gigaword)': glove,
    'FastText (Wiki News)': ft
}

results = []

for model_name, model in models.items():
    print(f"\nModelo: {model_name}")
    
    found_words = [w for w in vocab if w in model.key_to_index]
    missing_words = [w for w in vocab if w not in model.key_to_index]
    
    found_tokens = sum(vocab_freq[w] for w in found_words)
    total_tokens = sum(vocab_freq.values())
    
    coverage_vocab = len(found_words) / len(vocab) * 100
    coverage_tokens = found_tokens / total_tokens * 100
    
    print(f"Palabras encontradas: {len(found_words):,}/{len(vocab):,} ({coverage_vocab:.2f}%)")
    print(f"Tokens cubiertos: {found_tokens:,}/{total_tokens:,} ({coverage_tokens:.2f}%)")
    
    missing_freq = {w: vocab_freq[w] for w in missing_words}
    top_missing = sorted(missing_freq.items(), key=lambda x: x[1], reverse=True)[:10]
    
    print(f"\nTop 10 palabras NO encontradas (más frecuentes):")
    for word, freq in top_missing:
        print(f"  {word:20s}: {freq:5d} ocurrencias")
    
    results.append({
        'Modelo': model_name,
        'Cobertura Vocabulario (%)': round(coverage_vocab, 2),
        'Cobertura Tokens (%)': round(coverage_tokens, 2),
        'Palabras Encontradas': len(found_words),
        'Palabras No Encontradas': len(missing_words)
    })

# Tabla comparativa
print("\nTABLA COMPARATIVA DE COBERTURA")
results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))


Vocabulario total: 8,899 palabras únicas
Total de tokens: 126,935

Modelo: Word2Vec (Google News)
Palabras encontradas: 4,295/8,899 (48.26%)
Tokens cubiertos: 85,654/126,935 (67.48%)

Top 10 palabras NO encontradas (más frecuentes):
  <doc>               :  5810 ocurrencias
  <pat>               :  4895 ocurrencias
  a                   :  2101 ocurrencias
  to                  :  2080 ocurrencias
  and                 :  2024 ocurrencias
  of                  :  1580 ocurrencias
  yes,                :   577 ocurrencias
  guest_family:       :   559 ocurrencias
  no,                 :   509 ocurrencias
  i'm                 :   441 ocurrencias

Modelo: GloVe (Wiki Gigaword)
Palabras encontradas: 4,233/8,899 (47.57%)
Tokens cubiertos: 90,800/126,935 (71.53%)

Top 10 palabras NO encontradas (más frecuentes):
  <doc>               :  5810 ocurrencias
  <pat>               :  4895 ocurrencias
  yes,                :   577 ocurrencias
  guest_family:       :   559 ocurrencias
  no,       